## Hello World

The requirement is to use Convolutional Neural Network model to classify all given images into fake and original.

First, we need to import all the libraries that we are going to need to complete the task. As a nn library, we will be using pyTorch. Also, for the data preprocessing we are using numpy, and for the visualization we will be using pyplot.

In [1]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'torch'